In [1]:
import torch
import json
import pandas as pd
with open('../../meta/whisper_special_tokens.json') as f:
    SPECIAL_TOKENS = json.load(f)
LANG_TOKENS = SPECIAL_TOKENS['lang']
LANG_TOKEN_IDS = [lang_obj['id'] for lang_obj in LANG_TOKENS.values()]
MIN_LANG_ID = min(LANG_TOKEN_IDS)

In [2]:
lid_df = pd.DataFrame(
    {
        'id': lang_obj['id'],
        'rel_id': lang_obj['id']-MIN_LANG_ID,
        'lang': lang,
    } for lang, lang_obj in LANG_TOKENS.items()
)
lid_df

,id,rel_id,lang
0,50259,0,en
1,50260,1,zh
2,50261,2,de
3,50262,3,es
4,50263,4,ru
...,...,...,...
94,50353,94,ln
95,50354,95,ha
96,50355,96,ba
97,50356,97,jw


In [12]:
lid_logits_path = '../../data/tira-clean-split_lid_logits.pt'
lid_probs = torch.load(lid_logits_path)
len(lid_probs)

C:\Users\marks\AppData\Local\Temp\ipykernel_16920\3966952510.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lid_probs = torch.load(lid_logits_path)


99

In [13]:
lid_probs['en'].shape

torch.Size([16384])

In [14]:
prob_mat = torch.zeros(16384, 99)
prob_mat

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [15]:
for lang, prob_tensor in lid_probs.items():
    lang_id=LANG_TOKENS[lang]['id']-MIN_LANG_ID
    prob_mat[:,lang_id]=prob_tensor

In [16]:
lid_df['prob_mean']=prob_mat.mean(dim=0)
lid_df['prob_std']=prob_mat.std(dim=0)

In [18]:
lid_df.sort_values('prob_mean', ascending=False)[:10]

,id,rel_id,lang,prob_mean,prob_std
0,50259,0,en,0.079624,0.090021
3,50262,3,es,0.075314,0.143020
59,50318,59,sw,0.057751,0.140158
63,50322,63,si,0.051566,0.106907
13,50272,13,ar,0.048845,0.119238
37,50296,37,ml,0.043713,0.144792
22,50281,22,el,0.033359,0.127800
89,50348,89,tl,0.029087,0.077079
15,50274,15,it,0.028131,0.082246
40,50299,40,te,0.027548,0.077354
